In [13]:
import torch
import pyro
import pyro.distributions as dist
import pyro.optim as optim
from pyro.infer import SVI, Trace_ELBO, JitTrace_ELBO
import pandas as pd
from torch.distributions import constraints


In [3]:
dict_rest = pd.read_pickle("./data/crawled_data.pickle")

In [4]:
# organize data 
ids = [d["id"] for d in dict_rest]
names = [d["name"] for d in dict_rest]
lats = [d["coordinates"]["lat"] for d in dict_rest]
lngs = [d["coordinates"]["lng"] for d in dict_rest]
ratings = [ d["rating"] for d in dict_rest]
rating_ns = [ d["rating_n"] for d in dict_rest]
populartimes = [ d["populartimes"] for d in dict_rest]
poptime_sat = [ d[5]['data'] for d in populartimes]
poptime_sat_12 = [d[12] for d in poptime_sat]

In [47]:
selected_data = {'ids':ids,'names':names,'lats':lats,'lngs':lngs,\
                 'ratings':ratings,'rating_ns':rating_ns,'poptime_sat_12':poptime_sat_12}
df = pd.DataFrame(selected_data)
df_model = df[['ratings','poptime_sat_12']]
train_data = torch.tensor(df_model.values, dtype = torch.float)
rating, popularity = train_data[:, 0], train_data[:,1]

In [48]:
# #Tested whether SVI runs correctly, and it did.
# rating = torch.rand(30)
# test_dist = dist.Poisson(torch.exp(rating.sum() + 1))
# popularity = torch.tensor([test_dist.sample() for i in range(len(rating))], dtype = torch.float)

In [49]:
def model(rating, popularity):
    beta_0 = pyro.sample("beta_0", dist.Normal(0, 1))
    beta_1 = pyro.sample("beta_1", dist.Normal(0, 1))
    
    lambda_ = torch.exp(beta_0 + beta_1 * rating)
    
    with pyro.plate("data", len(popularity)):
        y = pyro.sample("obs", dist.Poisson(lambda_), obs=popularity)
    
    return y

In [50]:
def guide(rating, popularity):
    weights_loc = pyro.param('weights_loc', torch.randn(2))
    weights_scale = pyro.param('weights_scale', torch.ones(2), constraint=constraints.positive)        

    beta_0 = pyro.sample("beta_0", dist.Normal(weights_loc[0], weights_scale[0]))
    beta_1 = pyro.sample("beta_1", dist.Normal(weights_loc[1], weights_scale[1]))

    labmda_ = torch.exp(beta_0 * rating + beta_1)


In [51]:
svi = SVI(model, 
          guide, 
          optim.Adam({"lr": .005}), 
          loss=JitTrace_ELBO(), 
          num_samples=1000)

pyro.clear_param_store()
epoch = 100000
for i in range(epoch):
    ELBO = svi.step(rating, popularity)
    if i % 500 == 0:
        print(ELBO)
posterior = svi.run(rating, popularity)


/Users/wonjunson/anaconda3/envs/pp/lib/python3.7/site-packages/ipykernel_launcher.py:7: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  import sys


22376.296875
13588.4873046875
845555.25
85993.75
2229591.0
40585.44140625
232660.328125
12833.9296875
21935.009765625
10885.6376953125


KeyboardInterrupt: 